# 第5章 顧客の退会を予測する10本ノック (41~50)

In [1]:
# 下準備として，データのあるディレクトリに移動しておく
import os

DATA_ROOT_DIR = "./sample/5章/"
os.chdir(DATA_ROOT_DIR)

## ノック41 データを読み込んで利用データを成形しよう
今回は，データを読み込むだけでなく，データの成形も行う．

In [2]:
import pandas as pd
customer = pd.read_csv("customer_join.csv")
use_log_months = pd.read_csv("use_log_months.csv")

In [3]:
year_months = list(use_log_months["年月"].unique())
use_log = pd.DataFrame()

for i in range(1, len(year_months)):
    tmp = use_log_months.loc[use_log_months["年月"] == year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = use_log_months.loc[use_log_months["年月"] == year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    use_log = pd.concat([use_log, tmp], ignore_index=True)

use_log.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


## ノック42 退会前月の退会顧客データを作成しよう
このデータの場合，月末までに退会申請をすると，翌月に退会になる．そのため，退会の予測には，退会月の2ヵ月前のデータが必要になる．

In [4]:
from dateutil.relativedelta import relativedelta

exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])

for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)

# for文の中でうまくキャストされてない(バグかしら？)ので，手動でdatetime型にキャスト
exit_customer["exit_date"] = pd.to_datetime(exit_customer["exit_date"])

exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
use_log["年月"] = use_log["年月"].astype(str)
exit_use_log = pd.merge(use_log, exit_customer, on=["customer_id", "年月"], how="left")
print(len(use_log))
exit_use_log.head() 

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [5]:
exit_use_log = exit_use_log.dropna(subset=["name"])
print(len(exit_use_log))
print(len(exit_use_log["customer_id"].unique()))
exit_use_log.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


## ノック43 継続顧客のデータを作成しよう
継続顧客のデータを作成し，アンダーサンプリングを行う．

In [6]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_use_log = pd.merge(use_log, conti_customer, on=["customer_id"], how="left")
print(len(conti_use_log))
conti_use_log = conti_use_log.dropna(subset=["name"])
print(len(conti_use_log)) 

33851
27422


In [7]:
conti_use_log = conti_use_log.sample(frac=1).reset_index(drop=True)
conti_use_log = conti_use_log.drop_duplicates(subset="customer_id")
print(len(conti_use_log))
conti_use_log.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201902,PL552906,4,4.0,XXXXXX,C01,F,2015-12-01,NaN,CA1,...,オールタイム,10500.0,通常,4.583333,4.0,7.0,2.0,1.0,2019-04-30,40.0
2,201810,PL394207,4,7.0,XXXX,C01,F,2015-09-01,NaN,CA1,...,オールタイム,10500.0,通常,4.833333,5.0,7.0,2.0,1.0,2019-04-30,43.0
3,201805,OA862614,4,3.0,XXXX,C03,M,2015-10-01,NaN,CA1,...,ナイト,6000.0,通常,4.500000,4.0,7.0,3.0,1.0,2019-04-30,42.0
4,201811,TS600511,8,9.0,XX,C01,M,2017-11-01,NaN,CA1,...,オールタイム,10500.0,通常,7.000000,7.0,9.0,4.0,1.0,2019-04-30,17.0
5,201811,HI270880,3,3.0,XX,C02,F,2015-09-01,NaN,CA1,...,デイタイム,7500.0,通常,4.000000,3.0,7.0,1.0,1.0,2019-04-30,43.0


In [8]:
predict_data = pd.concat([conti_use_log, exit_use_log], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201902,PL552906,4,4.0,XXXXXX,C01,F,2015-12-01,NaN,CA1,...,10500.0,通常,4.583333,4.0,7.0,2.0,1.0,2019-04-30,40.0,NaT
1,201810,PL394207,4,7.0,XXXX,C01,F,2015-09-01,NaN,CA1,...,10500.0,通常,4.833333,5.0,7.0,2.0,1.0,2019-04-30,43.0,NaT
2,201805,OA862614,4,3.0,XXXX,C03,M,2015-10-01,NaN,CA1,...,6000.0,通常,4.500000,4.0,7.0,3.0,1.0,2019-04-30,42.0,NaT
3,201811,TS600511,8,9.0,XX,C01,M,2017-11-01,NaN,CA1,...,10500.0,通常,7.000000,7.0,9.0,4.0,1.0,2019-04-30,17.0,NaT
4,201811,HI270880,3,3.0,XX,C02,F,2015-09-01,NaN,CA1,...,7500.0,通常,4.000000,3.0,7.0,1.0,1.0,2019-04-30,43.0,NaT


## ノック44 予測する月の在籍期間を作成しよう
在籍期間を予想のための変数として用いる．

In [9]:
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])

deltas = [relativedelta(now, start) for now, start in zip(predict_data["now_date"], predict_data["start_date"])]
periods = [int(delta.years*12 + delta.months) for delta in deltas]

predict_data["period"] = periods
predict_data.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,now_date,period
0,201902,PL552906,4,4.0,XXXXXX,C01,F,2015-12-01,NaN,CA1,...,4.583333,4.0,7.0,2.0,1.0,2019-04-30,40.0,NaT,2019-02-01,38
1,201810,PL394207,4,7.0,XXXX,C01,F,2015-09-01,NaN,CA1,...,4.833333,5.0,7.0,2.0,1.0,2019-04-30,43.0,NaT,2018-10-01,37
2,201805,OA862614,4,3.0,XXXX,C03,M,2015-10-01,NaN,CA1,...,4.500000,4.0,7.0,3.0,1.0,2019-04-30,42.0,NaT,2018-05-01,31
3,201811,TS600511,8,9.0,XX,C01,M,2017-11-01,NaN,CA1,...,7.000000,7.0,9.0,4.0,1.0,2019-04-30,17.0,NaT,2018-11-01,12
4,201811,HI270880,3,3.0,XX,C02,F,2015-09-01,NaN,CA1,...,4.000000,3.0,7.0,1.0,1.0,2019-04-30,43.0,NaT,2018-11-01,38
